# chapter 08 | 텍스트 분석
#### 텍스트 분석
- 자연어처리 (Natural Language Processing, NLP) : 인간의 언어를 이해하고 해석
- 텍스트 마이닝 (Text Mining) : 비정형 데이터에서 의미있는 정보를 추출
    - 텍스트 분류 (Text Classification, Text Categorization) : 문서가 특정 분류 또는 카테고리에 속하는 것을 예측하는 기법. (예) 스팸 메일 검출, 지도학습 
    - 감성 분석 (Sentiment Analysis) : 텍스트에 나타나는  : 감정/의견/기분 등 주관적 요소를 분석. (예)소셜미디어 감정 분석, 영화나 제품에 대한 긍정 또는 리뷰
    - 텍스트 요약 (Summarization) : 텍스트 내에서 중요한 주제나 중심 사상을 추출하는 기법. (예) 토픽 모델링
    - 텍스트 군집화 (Clustering), 유사도 측정 : 비슷한 유형의 문서에 대해 군집화를 수행하는 기법. 텍스트 분류를 비지도학습으로 수행하는 방법의 일환. 

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 01. 텍스트 분석 이해

- 텍스트는 비정형 데이터, 텍스트 데이터를 어떻게 피처 형태로 추출하고 추출된 피처에 의미있는 값을 부여하는가 하는 것이 매우 중요
- 피처 벡터화(Feature Vectorization) = 피처 추출(Feature Extraction) : 
    - BOW (Bag of Words)
    - Word2Vec
    
### 텍스트 분석 수행 프로세스
1. __텍스트 사전 준비작업(텍스트 전처리)__ :  대/소문자 변경, 특수문자 삭제 등 텍스트 클렌징, 단어(Words) 등의 토큰화 작업, 불용어(Stop Words) 제거, 어근 추출(Stemming/Lemmatization) 등 텍스트 정규화 작업을 통칭
2. __피처 벡터화/추출__ : 사전 준비 작업으로 가공된 텍스트에서 피처를 추출하고 여기에 벡터 값을 할당, BOW와 Word2Vec 방식이 있으며, BOW는 대표적으로 Count 기밥과 TF-IDF 기반 벡터화가 있음
3. __ML 모델 수립 및 학습/예측/평가__ : 피처 벡터화된 데이터 세트에 ML 모델을 적용해 학습/에측 및 평가를 수행

### 파이썬 기반 NLP, 텍스트 분석 패키지
- NLTK : 파이썬 대표적인 NLP 패키지, 수행 속도에 아쉬움이 있어서 실제 대량의 데이터 기반에서는 제대로 활용 X
- GEnsim : 토픽 모델링
- SpaCy 

## 02. 텍스트 사전 준비 작업(텍스트 전처리) - 텍스트 정규화

### 클렌징
텍스트 분석에 방해가 되는 불필요한 문자, 기호 등을 사전에 제거

### 텍스트 토큰화
- 문장 토큰화 : 문서에서 문장을 분리
- 단어 토큰화 : 문장에서 단어를 분리

#### 문장 토큰화 (Sentence Tokenization) 
- 문장의 마침표(.), 개행문자(\n) 등 문장의 마지막을 뜻하는 기호 또는 정규표현식에 따라 분리
- `nltk.tokenize.sent_tokenize(text, language='english')` 사용

In [5]:
from nltk import sent_tokenize
import nltk
# nltk.download('punkt') # 마침표, 개행문자 다운로드

text_sample = 'The Matrix is everywhere its all around us, here even in this room. \
               You can see it out your window or on your television. \
               You feel it when you go to work, or go to church or pay your taxes.'

sentences = sent_tokenize(text_sample)
print(len(sentences), type(sentences))
print(sentences)

3 <class 'list'>
['The Matrix is everywhere its all around us, here even in this room.', 'You can see it out your window or on your television.', 'You feel it when you go to work, or go to church or pay your taxes.']


#### 단어 토큰화 (Word Tokenization)
- 공백, 콤마(,), 마침표(.), 개행문자, 정규표현식 등으로 단어를 분리

In [6]:
from nltk import word_tokenize

sentence = "The Matrix is everywhere its all around us, here even in this room."
words = word_tokenize(sentence)
print(len(words), type(words))
print(words)

15 <class 'list'>
['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


#### tokenize_text()
여러 개의 문장으로 된 입력 데이터를 문장별로 단어 토큰화하게 만드는 함수 생성


In [13]:
from nltk import word_tokenize, sent_tokenize

def tokenize_text(text):
    sents = sent_tokenize(text)
    word_tokens = [word_tokenize(sent) for sent in sents]
    return word_tokens

word_tokens = tokenize_text(text_sample)
print(type(word_tokens), len(word_tokens))
print(word_tokens)

<class 'list'> 3
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


단어 하나씩 토큰화할 경우에 문맥적인 의미가 무시되기 때문에 __n-gram__ 을 사용  
__n_gram__ : 연속된 n개의 단어를 하나씩 토근화 단위로 분리  
(예) "Agent Smith knocks the door" → (Agent, Smith), (Smith, knocks), (knocks, the), (the, door)

### 불용어(Stop Words) 제거
불용어 : 필수 문법 요소지만 큰 의미가 없는 단어 (예) is, the, a

In [8]:
import nltk
# nltk.download('stopwords') # 불용어 다운로드

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gimdonghwa/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
print(len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english'))

179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

In [19]:
print(word_tokens)

[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


In [20]:
stopwords = nltk.corpus.stopwords.words('english')

all_tokens = []
for sent in word_tokens :
    filter_words = []
    for word in sent :
        if word not in stopwords :
            word = word.lower()
            filter_words.append(word)
    all_tokens.append(filter_words)
            
print(type(all_tokens), len(all_tokens))
print(all_tokens)

<class 'list'> 3
[['the', 'matrix', 'everywhere', 'around', 'us', ',', 'even', 'room', '.'], ['you', 'see', 'window', 'television', '.'], ['you', 'feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


### Stemming과 Lemmatization
- 문법적 또는 의미적으로 변화하는 단어의 원형을 찾는 것
- Lemmatization이 Stemming보다 더 정교하며 의미론적인 기반에서 단어의 원형을 찾음

1. Stemming : 원형 단어로 변환시 일반적인 방법을 적용하거나 더 단순화된 방법을 적용해 원래 단어에서 일부 철자가 훼손된 어근 단어를 추출하는 경향 O
    - 다양한 Stemmer : Porter, Lancaster, Snowball, Stemmer
    - `LancasterStemmer`
2. Lemmatization : 품사와 같은 문법적인 요소와 더 의미적인 부분을 감안해 정확한 철자로 된 어근 단어를 찾음, 더 오랜 시간 걸림
    - `WordNetLemmatizer`


In [24]:
from nltk.stem import LancasterStemmer

stemmer = LancasterStemmer()

print(stemmer.stem("worked"), stemmer.stem("works"), stemmer.stem("working"))
print(stemmer.stem("amused"), stemmer.stem("amusing"), stemmer.stem("amuses"))
print(stemmer.stem("fancier"), stemmer.stem("fancy"), stemmer.stem("fanciest"))

work work work
amus amus amus
fant fant fanciest


- 동사의 시제를 나타내는 어미 '-ed','-ing', '-s/es'가 붙은 단어 → 기본 단어 잘 찾아냄
- 형용사 규칙에 따라 비교형, 최상급형인 단어 → 기본 단어 잘 찾아냄

```
nltk.download('wordnet') 
```
wordnet : [링크](https://rfriend.tistory.com/546)

In [27]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gimdonghwa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
lemma = WordNetLemmatizer()
print(lemma.lemmatize('amused','v'), lemma.lemmatize('amusing','v'))
print(lemma.lemmatize('happier','a'), lemma.lemmatize('happiest','a'))

amuse amuse
happy happy


## 03. Bag of Words - BOW
- 문서 내 모든 단어를 한꺼번에 봉투 안에 넣은 뒤에 흔들어 섞는다는 의미
- 단어 수 기반으로 피처 추출
<img src="https://hleecaster.com/wp-content/uploads/2020/01/bow01.jpg">

#### 장점 
쉽고 빠른 구축

#### 단점
1. 문맥 의미(senmantic Context) 반영 부족 : 문장의 순서를 고려 X → 단어의 문맥적인 의미 무시 → n_gram 사용하지만 제한적
2. 희소 행렬 문제 (희소성, 희소 행렬) : ML 알고리즘의 수행 시간과 예측 성능을 떨어트림

cf. 희소 행렬(Sparse Matrix) : 대규모의 칼럼으로 구성된 행렬에서 대부분의 값이 0으로 채워지는 행렬   
(반) 밀집 행렬(Dense Matrix) : 대부분의 값이 0이 아닌 의미있는 값으로 채워져 있는 행렬

### BOW 피처 벡터화
- 피처 벡터화 : 텍스트를 특정 의미를 가지는 숫자형 값인 벡터 값으로 변환하는 과정  
(예) 문서의 텍스트를 단어로 추출해 피처로 할당하고, 문서 내에 각 단어의 발생 빈도와 같은 값을 피처에 값으로 부여해 벡터화, M개의 문서, N개의 독립적인 단어라면 M X N 단어 피처로 이루어진 행렬을 구성

#### 1. 카운트 기반 벡터화
단어 피처에 값을 부여할 때 해당 단어가 나타나는 횟수를 부여하는 방식
→ 문제 : 문서의 특징 X, 언어의 특성상 문장에서 자주 나타나는 단어까지 높은 값을 부여하게 됨

#### 2. TF-IDF(Term Frequency - Inverse Document Frequency) 기반의 벡터화
개별 문서에서 자주 나타나는 단어에 높은 가중치를 주되, 모든 문서에서 전반적으로 자주 나타나는 문서에 대해서는 페널티를 주는 방식으로 값을 부여  
어떤 문서에서 특정 단어가 자주 나타나면 그 단어는 해당 문서를 특징짓는 중요 단어일 수 있지만, 그 단어가 다른 문서에도 자주 나타난다면 언어 특성상 범용적으로 자주 사용되는 단어일 가능성이 높음

$$ TFIDF_{i} = TF_{i} \times log \frac{N}{DF_{i}}$$
$$ where\  TF_{i} = 개별\ 문서에서의\ 단어\ 빈도, DF_{i}\ =\ 단어\ i를\ 가지고\ 있는\ 문서\ 개수, N\ =\ 전체\ 문서\ 개수$$ 

### 사이킷런의 Count 및 TF-IDF 벡터화 구현 : CountVectorizer, TfidfVectorizer

#### CountVectorizer()
- 소문자 일괄 변환, 토큰화, 불용어 필터링 등 텍스트 전처리도 함께 수행
- fit(), transform() 메서드를 통해 피처 벡터화된 객체를 반환

```
class sklearn.feature_extraction.text.CountVectorizer(*, input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern='(?u)\b\w\w+\b', ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.int64'>)
```

<U> 주요인자 </U>
- `max_df` : 전체 문서에 걸쳐서 너무 높은 빈도수를 가지는 단어 피처를 제외하기 위한 인자
- `min_df` : 전체 문서에 걸쳐서 너무 낮은 빈도수를 가지는 단어 피처를 제외하기 위한 인자
- `max_fetures` : 추출하는 피처의 개수를 제한, 가장 높은 빈도를 가지는 단어 순으로 정렬해 선택
- `stop_words` " 'english'로 지정하면 영어의 스톱 워드로 지정된 단어는 추출에서 제외 (*다른 언어 지원 X, 내부적으로 전처리 수행한다는 것을 알 수 있음*)
- `n_gram_range` : 모델의 단어 순서를 보강하기 위한 n_gram 범위를 설정 튜플 형태로 (범위 최솟값, 범위 최댓값)을 지정 (예) (1,2) :  토큰화된 단어를 1개씩, 순서대로 2개씩 묶어서 피처로 추출
- `analyzer` (default='word'): 피처 추출을 수행하는 단위 (이 외에 'character')
- `token_pattern` : 토큰화를 수행하는 정규 표현식 패턴을 지정
- `tokenizer` : 토큰화를 별도의 커스텀 함수로 이용시 적용 



### BOW 벡터화를 위한 희소 행렬
- BOW 형태를 가진 언어 모델의 피처 벡터화는 대부분 희소 행렬
- 불필요한 0 할당, 메모리 공간 소모 ↑, 연산 시간 ↑  
→ 적은 메모리 공간을 차지할 수 있도록 변환 : __COO 형식, CSR 형식__

### 희소 행렬 - COO(Coordinates) 형식
0이 아닌 데이터만 별도의 데이터 배열로 저장하고, 그 데이터가 가리키는 행과 열의 위치를 별도의 배열로 저장하는 방식
(예) 2차원 데이터 : [[3,0,1],[0,2,0]]   
→ data_array = [3,1,2]), row_array = [0,0,1], col_array = [0,2,1]

In [8]:
from scipy.sparse import coo_matrix

data = np.array([3,1,2])
row_array = np.array([0,0,1])
col_array = np.array([0,2,1])

coo_matrix((data, (row_array, col_array))).toarray()

array([[3, 0, 1],
       [0, 2, 0]])

### 희소행렬 - CSR(Compressed Spare Row) 형식
행과 열의 위치를 나타낼 때 반복적인 위치 데이터를 사용해야하는 문제점을 해결한 방식
 
 
<img src='https://t1.daumcdn.net/cfile/tistory/99C845445F69BC8A32' width='80%'>

In [20]:
from scipy.sparse import coo_matrix, csr_matrix

dense2 = np.array([[0,0,1,0,0,5],
             [1,4,0,3,2,5],
             [0,6,0,3,0,0],
             [2,0,0,0,0,0],
             [0,0,0,7,0,8],
             [1,0,0,0,0,0]])

data2 = np.array([1, 5, 1, 4, 3, 2, 5, 6, 3, 2, 7, 8, 1])

row_pos = np.array([0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 5])
col_pos = np.array([2, 5, 0, 1, 3, 4, 5, 1, 3, 0, 3, 5, 0])

row_pos_idx = np.array([0, 2, 7, 9, 10, 12, 13])

print("coo_matrix : \n", coo_matrix((data2, (row_pos, col_pos))).toarray(), '\n')
print("csr_matrix with the same input as coo_matrix : \n", csr_matrix((data2, (row_pos, col_pos))).toarray(), '\n') # csr_matrix가 coo_matrix와 동일하게 작동
print("csr_matrix : \n", csr_matrix((data2, col_pos, row_pos_idx)).toarray())

coo_matrix : 
 [[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]] 

csr_matrix with the same input as coo_matrix : 
 [[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]] 

csr_matrix : 
 [[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


#### 참고 문헌 
- [Python] Numpy 희소행렬을 SciPy 압축 희소 열 행렬 (Compressed sparse row matrix)로 변환하기 [링크](https://rfriend.tistory.com/551)
- Bag-of-Words(BoW) 쉽게 이해하기 [링크](https://hleecaster.com/nlp-bag-of-words-concept/)